In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_head = 4
dropout = 0.2

# ------------

torch.manual_seed(2023)

# Load the Victor Hugo dataset
with open('data/hugo_contemplations.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        # YOUR CODE
        C = n_embd
        # add you key, query and value definitions
        # define the Key layer  
        self.key = nn.Linear(C, head_size)
        # define the Query layer
        self.query = nn.Linear(C, head_size)
        # define the Value layer
        self.value =  nn.Linear(C, head_size)
        ###
        self.dropout = nn.Dropout(dropout) 
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))



    def forward(self, x):
        B,T,C = x.shape
        ## YOUR CODE HERE
        # apply each layer to the input
        k = self.key(x) # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)
        v = self.value(x) # (B, T, head_size)
        # compute the normalize product between Q and K
        weights = q @ k.transpose(1,2) # (B, T, head_size) @ (B, 16, head_size) -> (B, T, T)
        # apply the mask (lower triangular matrix)
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        # apply the softmax
        weights = nn.functional.softmax(weights, dim=-1)
        # apply the dropout
        weights = self.dropout(weights)
        out = weights @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        ## YOUR CODE HERE
        ## list of num_heads modules of type Head
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.dropout = nn.Dropout(dropout)
        ###
        
    def forward(self, x):
        ## YOUR CODE HERE
        ## apply each head in self.heads to x and concat the results 
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(out)
        return out

class FeedForward(nn.Module):
    """ a simple MLP with RELU """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU(),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ A single bloc of multi-head attention """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.ln1 = nn.LayerNorm(n_embd)
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ln2 = nn.LayerNorm(n_embd)
        self.ffwd = FeedForward(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))  
        x = x + self.ffwd(self.ln2(x)) 
        return x
        
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.b1 = Block(n_embd, n_head)
        self.b2 = Block(n_embd, n_head)
        self.b3 = Block(n_embd, n_head)
        self.ln = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.ln(x)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


# generate from the model
prompt = torch.tensor(encode(['\n']))
context = torch.ones((1,1), dtype=torch.long, device=device)*prompt
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


0.019941 M parameters
step 0: train loss 4.7806, val loss 4.7944
step 500: train loss 2.4677, val loss 2.5455
step 1000: train loss 2.3054, val loss 2.3337
step 1500: train loss 2.2320, val loss 2.2529
step 2000: train loss 2.1850, val loss 2.1776
step 2500: train loss 2.1402, val loss 2.1361
step 3000: train loss 2.1283, val loss 2.1140
step 3500: train loss 2.1055, val loss 2.1004
step 4000: train loss 2.0843, val loss 2.0768
step 4500: train loss 2.0775, val loss 2.0528
step 4999: train loss 2.0638, val loss 2.0422

Si vingare méséla que les souts. de
Pans l'étontila sumormaime eienchsenour.
Où l'acar que sors la vess détes vour d'ul, des s'itses enue et fl'ine;
Jes ves es re, êve, on, on des-muais pouTon a sords, ubret r'ilcre,.
Le ses, re poivernil quu'émère qui de mêle le rous laps noit ibrendi du flus sombriEtand sois mare furiarièt enc d'oubes sobrir.
Les fe inchis, chre voutant misez! estrané;
    
Vol sa pavent vire l'et me surofête!
Certénoidre, fantonge, lu fonitent!
Dès l'